# Evaluate RAG answer quality

## Setup API clients

In [1]:
import os

# import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
import json

dotenv.load_dotenv()

# azure_credential = azure.identity.DefaultAzureCredential()

# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_OPENAI_SERVICE = "ragsearchpocopenai"
AZURE_OPENAI_ADA_DEPLOYMENT = "text-embedding-ada-002"

# azure_credential = azure.identity.DefaultAzureCredential()
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
# token_provider = azure.identity.get_bearer_token_provider(azure_openai_key,
#     "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    api_key=azure_openai_key,
)


def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(
        model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text
    )
    return get_embeddings_response.data[0].embedding


# Initialize Azure search client
AZURE_SEARCH_SERVICE = "ragsearchpocsch"
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_FULL_INDEX = "ragsearch"
search_client = SearchClient(
    AZURE_SEARCH_ENDPOINT,
    AZURE_SEARCH_FULL_INDEX,
    credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY")),
)

## Get answer for a question

In [3]:
user_question = "What does a product manager do?"
user_question_vector = get_embedding(user_question)

r = search_client.search(
        user_question,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="vector")],
        query_type="semantic",
        semantic_configuration_name="ragsearch-semantic-config")

sources = "\n\n".join([f"[{doc['title']}]: {doc['chunk']}\n" for doc in r])


SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


answer = response.choices[0].message.content
print(answer)

A product manager is responsible for overseeing product development and marketing strategy, ensuring successful product launches, and implementing product life-cycle management processes. They collaborate with internal teams such as engineering, sales, marketing, and finance, and analyze industry trends to identify opportunities for new products. Key responsibilities include developing product marketing plans, managing product budgets, and monitoring product performance and customer feedback [role_library.pdf].


## Evaluate the answer quality

We can use the `promptflow-evals` package to run GPT-based evaluators on the RAG responses.

In [ ]:
import os

from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.evals.evaluators import QAEvaluator

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_deployment="gpt-4o-mini",
    api_key=os.getenv("AZURE_OPENAI_KEY"),
)

eval_fn = QAEvaluator(model_config)
qa_score = eval_fn(
    question=user_question,
    answer=answer,
    context=sources,
    ground_truth="Product managers are responsible for the strategy, roadmap, and feature definition of a product or product line. They are the key point of contact between the engineering team and other departments, such as marketing and sales. They ensure that the product meets customer needs and aligns with business goals.",
)
print(qa_score)

{'f1_score': 0.380952380952381, 'gpt_similarity': 4.0, 'gpt_groundedness': 5.0, 'gpt_coherence': 5.0, 'gpt_relevance': 5.0, 'gpt_fluency': 5.0}
